In [5]:
import cv2
import os

input_folder = "./data"
output_folder = "./data"

os.makedirs(output_folder, exist_ok=True)  # Ensure output directory exists

for filename in os.listdir(input_folder):
    if filename.endswith(".png"):  
        img_path = os.path.join(input_folder, filename)
        
        # Read the image (forcefully add an alpha channel if missing)
        img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
        
        if img.shape[2] == 3:  # If no alpha channel, add one
            b, g, r = cv2.split(img)
            a = np.ones_like(b) * 255  # Create a full white alpha channel
            img = cv2.merge((b, g, r, a))

        # Save image
        cv2.imwrite(os.path.join(output_folder, filename), img)
        print(f"Converted {filename} to RGBA.")

Converted 03.png to RGBA.
Converted 05.png to RGBA.
Converted 02.png to RGBA.
Converted 01.png to RGBA.
Converted 08.png to RGBA.
Converted 10.png to RGBA.
Converted 07.png to RGBA.
Converted 04.png to RGBA.
Converted 06.png to RGBA.
Converted 09.png to RGBA.


In [6]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import decode_predictions
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
import tensorflow as tf
import os
import numpy as np

In [13]:
def image_files(path):
    '''
    Creates a list with the names of the images to be processed.
    '''
    image_files = []
    # creates a ScandirIterator aliased as files
    with os.scandir(path) as files:
        for file in files:
            if file.name.endswith('.jpg'):
                # adds only the image files to the list
                image_files.append(file.name)
    return sorted(image_files)


def load_model():
    model = VGG16()
    model = Model(inputs = model.inputs, outputs = model.output)
    return model


def recognize_objects(file, path, model):
    '''
    Recognizes objects in an image file using the VGG16 model.
    '''
    # load the image as a 224x224 array
    img = load_img(f'{path}/{file}', target_size=(224,224))
    # convert from 'PIL.Image.Image' to numpy array
    img = np.array(img, dtype=np.float32)
    # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels)
    reshaped_img = img.reshape(1,224,224,3)
    # prepare image for model
    imgx = preprocess_input(reshaped_img)
    imgx = tf.convert_to_tensor(imgx)
    # get object names
    objects = model.predict(imgx)
    return objects

In [12]:
path = "./data"
model = load_model()
#print(model.input_shape)  # Expected input shape

files = image_files(path)

for file in files: 
    preds= recognize_objects(file, path, model)
    top_preds = decode_predictions(preds, top=5)[0]

    print(f"Objects detected in {file} are:")
    for i, (imagenet_id, label, score) in enumerate(top_preds):
        print(f"{i+1}. {label} ({score*100:.2f}%)")
    print()
       

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step


/home/rallypal/.pyenv/versions/lewagon/lib/python3.10/site-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_92']
Received: inputs=Tensor(shape=(1, 224, 224, 3))
  warnings.warn(msg)


Objects detected in 01.jpg are:
1. crutch (7.83%)
2. swab (7.47%)
3. drum (6.01%)
4. panpipe (5.55%)
5. sombrero (3.34%)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
Objects detected in 02.jpg are:
1. fountain (15.15%)
2. swab (6.54%)
3. crutch (6.14%)
4. umbrella (5.21%)
5. trench_coat (4.38%)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
Objects detected in 03.jpg are:
1. trench_coat (9.80%)
2. hair_spray (9.43%)
3. abaya (8.95%)
4. academic_gown (7.39%)
5. cleaver (7.20%)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
Objects detected in 04.jpg are:
1. library (22.30%)
2. gondola (19.80%)
3. bookshop (18.81%)
4. streetcar (16.26%)
5. prison (2.46%)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
Objects detected in 05.jpg are:
1. obelisk (16.47%)
2. church (9.61%)
3. beacon (9.59%)
4. flagpole (8.93%)
5. megalith (7.47%)

